# GOME-2 functions

In [ ]:
def GOME_version(component_nom, year, month):

    """ Get version of GOME-2 dataset for each component nomenclature

         Args:
            component_nom (str): Component chemical nomenclature
            year (str): Year of dataset
            month (str): Month of dataset

        Returns:
            version (str): GOME-2 dataset version
    """
    
    year = int(year)
    month = int(month)
    
    if component_nom == 'NO2':
        version = 'v1'

    if component_nom == 'O3':
        if (year <= 2016 and month <= 10) or (year <= 2015):
            version = 'v1'
        else:
            version = 'v1.1'

    return version

In [ ]:
def GOME_download(component_nom, date):

    """ Download the GOME-2 with wget

         Args:
            component_nom (str): Component chemical nomenclature
            date (str): Query month as date

        Returns:
            product_name (str): Product name of GOME-2 product
    """

    year = date.split('-')[0]
    month = date.split('-')[1]
    version = GOME_version(component_nom, year, month)

    if component_nom == 'O3':
        product_name = ''.join(['GOME_', component_nom, 'tropo_Tropics_', year, month, '_METOPB_DLR_', version, '.nc'])
        path = 'ftp://acsaf.eoc.dlr.de/gome2b/level3/' + component_nom + 'tropo' + '/' + year + '/' + product_name
        
    else:
        product_name = ''.join(['GOME_', component_nom, '_Global_', year, month, '_METOPB_DLR_', version, '.nc'])
        path = 'ftp://acsaf.eoc.dlr.de/gome2b/level3/' + component_nom + '/' + year + '/' + product_name

    url = 'data/gome/' + component_nom + '/' + product_name

    print(f'Downloading {product_name}...')
    subprocess.run(['wget', '-nc', path, '-O', url])

    return product_name

In [ ]:
def GOME_read(product_names, component_nom, sensor_column):

    """ Read the GOME-2 dataset as xarray dataset object and assign time

         Args:
            product_names (arr): Product names of GOME-2 products
            component_nom (str): Component chemical nomenclature
            sensor_column (str): Name of sensor column in downloaded dataset

        Returns:
            sensor_ds (xarray): GOME-2 dataset in xarray format
            sensor_type (str): Sensor type ('NRT')
    """

    sensor_ds_all = []

    for product_name in product_names:

        sensor_ds = xr.open_dataset('data/gome/' + component_nom + '/' + product_name)
        sensor_ds_int = xr.open_dataset('data/gome/' + component_nom + '/' + product_name, group = 'PRODUCT')
        sensor_ds[sensor_column] = sensor_ds_int[sensor_column]

        # Add datetime (e.g.: for June 2021, 2021-06-01)
        if component_nom == 'O3':
            year = int(product_name.split('Tropics_')[1].split('_METOPB')[0][0:4])
            month = int(product_name.split('Tropics_')[1].split('_METOPB')[0][4:6])
        else:
            year = int(product_name.split('Global_')[1].split('_METOPB')[0][0:4])
            month = int(product_name.split('Global_')[1].split('_METOPB')[0][4:6])
        time_str = datetime.datetime(year, month, 1)
        sensor_ds = sensor_ds.assign_coords({'time': time_str}).expand_dims(dim = ['time'])

        sensor_ds_all.append(sensor_ds)
    
        # Merge if there is more than one product
        if len(product_names) >= 2:

            sensor_ds = xr.merge(sensor_ds_all)

    sensor_type = 'L3'

    return sensor_ds, sensor_type